In [21]:
## 4.17 Solutions

import numpy as np
from triang import backsub, testcreate, testsolve

# 1. Gaussian elimination WITHOUT pivoting
def gauelim(inA, inbs):
    A = np.copy(inA)
    bs = np.copy(inbs)
    n = bs.size

    for j in range(n-1):
        for i in range(j+1, n):
            coeff = A[i,j] / A[j,j]
            A[i,j:] -= coeff * A[j,j:]
            bs[i] -= coeff * bs[j]

    xs = backsub(A, bs)
    return xs

# 2. Gaussian elimination WITH partial pivoting
def gauelim_pivot(inA, inbs):
    A = np.copy(inA)
    bs = np.copy(inbs)
    n = bs.size

    for j in range(n-1):
        # Partial pivoting: find row with maximum element in current column
        k = np.argmax(np.abs(A[j:, j])) + j
        
        if k != j:
            # Swap rows
            A[j, :], A[k, :] = A[k, :].copy(), A[j, :].copy()
            bs[j], bs[k] = bs[k], bs[j]
            print(f"  Pivot: swapped rows {j} and {k}")

        for i in range(j+1, n):
            coeff = A[i, j] / A[j, j]
            A[i, j:] -= coeff * A[j, j:]
            bs[i] -= coeff * bs[j]

    xs = backsub(A, bs)
    return xs

# 3. Gaussian elimination with SMALL PIVOT REPLACEMENT
def gauelim_replace(inA, inbs, epsilon=1e-20):
    A = np.copy(inA)
    bs = np.copy(inbs)
    n = bs.size

    for j in range(n-1):
        # Check if pivot is zero and replace if necessary
        if abs(A[j, j]) < 1e-15:
            print(f"  Replacement: pivot A[{j},{j}] = {A[j,j]:.2e} replaced with {epsilon:.2e}")
            A[j, j] = epsilon
        
        for i in range(j+1, n):
            coeff = A[i, j] / A[j, j]
            A[i, j:] -= coeff * A[j, j:]
            bs[i] -= coeff * bs[j]

    xs = backsub(A, bs)
    return xs

# Test the given problem
def main():    
    # Given matrix and vector from the problem
    A = np.array([4., 4, 8, 4, 4, 5, 3, 7, 8, 3, 9, 9, 4, 7, 9, 5]).reshape(4,4)
    bs = np.array([1., 2, 3, 4])
    
    print("Matrix A:")
    print(A)
    print(f"\nRight-hand side b: {bs}")
    
    print("1. GAUSSIAN ELIMINATION WITHOUT PIVOTING")
    xs1 = gauelim(A, bs)
    residual1 = np.linalg.norm(A @ xs1 - bs)
    print(f"Solution: {xs1}")
    
    print("2. GAUSSIAN ELIMINATION WITH PARTIAL PIVOTING")
    xs2 = gauelim_pivot(A, bs)
    residual2 = np.linalg.norm(A @ xs2 - bs)
    print(f"Solution: {xs2}")
   
    print("3. GAUSSIAN ELIMINATION WITH PIVOT REPLACEMENT (10^-20)")
    xs3 = gauelim_replace(A, bs, 1e-20)
    residual3 = np.linalg.norm(A @ xs3 - bs)
    print(f"Solution: {xs3}")
   
if __name__ == '__main__':
    main()

Matrix A:
[[4. 4. 8. 4.]
 [4. 5. 3. 7.]
 [8. 3. 9. 9.]
 [4. 7. 9. 5.]]

Right-hand side b: [1. 2. 3. 4.]
1. GAUSSIAN ELIMINATION WITHOUT PIVOTING
Solution: [nan nan inf inf]
2. GAUSSIAN ELIMINATION WITH PARTIAL PIVOTING
  Pivot: swapped rows 0 and 2
  Pivot: swapped rows 1 and 3
  Pivot: swapped rows 2 and 3
Solution: [ nan  inf -inf -inf]
3. GAUSSIAN ELIMINATION WITH PIVOT REPLACEMENT (10^-20)
Solution: [nan nan inf inf]


Problem 4.17: Answers to Questions

1. Does pivoting help in any way?
Yes, but only to reveal the problem.

Without pivoting:
- Gaussian elimination fails immediately with division by zero
- Algorithm crashes at the first zero pivot

With pivoting:
- Prevents division-by-zero errors
- Allows algorithm to complete
- Reveals the true nature of the problem through `-inf`/`inf` results
- Shows that the matrix is singular and system is inconsistent

2. What are your conclusions about this matrix?
The matrix is SINGULAR and the system is INCONSISTENT.

- Ax = b has no solution and is mathematically impossible
- The system is inconsistent
- "Negative inf inf" results confirm numerical instability due to singularity

3. Does replacing zero pivots with 10⁻²⁰ work?
No, it produces numerically meaningless results. It solves a different problem (A'x = b where A' ≠ A). It masks the fundamental singularity issue and produces solutions with enormous residual errors.

In [27]:
import numpy as np
from triang import testcreate, testsolve

def jacobi_cyclic(n, kmax=1000, tol=1.e-5):
    # Initialize solution vector
    xnews = np.zeros(n)
    
    print(f"Jacobi iterative method for n={n}")
    print("Tolerance: 1e-5")
    print("Iteration  Solution vector (first 5 elements)          Max Error")
    
    for k in range(1, kmax + 1):
        xs = np.copy(xnews)  # Store previous iteration
        
        # Jacobi iteration for cyclic tridiagonal matrix
        # Equation 1: x₀ = (1 + x₁ + xₙ₋₁) / 4
        xnews[0] = (1 + xs[1] + xs[n-1]) / 4
        
        # Equations 2 to n-1: xᵢ = (1 + xᵢ₋₁ + xᵢ₊₁) / 4
        for i in range(1, n-1):
            xnews[i] = (1 + xs[i-1] + xs[i+1]) / 4
        
        # Equation n: xₙ₋₁ = (1 + x₀ + xₙ₋₂) / 4
        xnews[n-1] = (1 + xs[0] + xs[n-2]) / 4
        
        # Check convergence using maximum relative error
        max_err = 0.0
        for i in range(n):
            if abs(xnews[i]) > 1e-15:  # Avoid division by zero
                err = abs((xnews[i] - xs[i]) / xnews[i])
                if err > max_err:
                    max_err = err
            else:
                err = abs(xnews[i] - xs[i])
                if err > max_err:
                    max_err = err
        
        # Print progress
        if k <= 10 or k % 100 == 0 or max_err < tol:
            # Show first 5 elements of solution vector
            first_five = [f"{x:.6f}" for x in xnews[:5]]
            print(f"{k:4d}    [{', '.join(first_five)}, ...]    {max_err:.2e}")
        
        if max_err < tol:
            print(f"Convergence achieved after {k} iterations")
            print(f"Final error: {max_err:.2e}")
            break
    else:
        print(f"✗ Maximum iterations ({kmax}) reached without convergence")
        print(f"✗ Final error: {max_err:.2e}")
    
    return xnews


if __name__ == '__main__':
    print("Jacobi Method for Cyclic Tridiagonal System")
    
    n_values = [10, 20]
    
    for n in n_values:
        print("\n" + "=" * 80)
        print(f"SOLVING FOR n = {n}")
        print("=" * 80)
        
        solution = jacobi_cyclic(n, kmax=1000, tol=1.e-5)

Jacobi Method for Cyclic Tridiagonal System

SOLVING FOR n = 10
Jacobi iterative method for n=10
Tolerance: 1e-5
Iteration  Solution vector (first 5 elements)          Max Error
   1    [0.250000, 0.250000, 0.250000, 0.250000, 0.250000, ...]    1.00e+00
   2    [0.375000, 0.375000, 0.375000, 0.375000, 0.375000, ...]    3.33e-01
   3    [0.437500, 0.437500, 0.437500, 0.437500, 0.437500, ...]    1.43e-01
   4    [0.468750, 0.468750, 0.468750, 0.468750, 0.468750, ...]    6.67e-02
   5    [0.484375, 0.484375, 0.484375, 0.484375, 0.484375, ...]    3.23e-02
   6    [0.492188, 0.492188, 0.492188, 0.492188, 0.492188, ...]    1.59e-02
   7    [0.496094, 0.496094, 0.496094, 0.496094, 0.496094, ...]    7.87e-03
   8    [0.498047, 0.498047, 0.498047, 0.498047, 0.498047, ...]    3.92e-03
   9    [0.499023, 0.499023, 0.499023, 0.499023, 0.499023, ...]    1.96e-03
  10    [0.499512, 0.499512, 0.499512, 0.499512, 0.499512, ...]    9.78e-04
  17    [0.499996, 0.499996, 0.499996, 0.499996, 0.499996, ...